# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [1]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 4개의 권한 추가
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
    - AWSCloud9SSMAccessRole

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

- #### **Please change to `install_complex_pdf = True` when utilizing Complex PDF**.
    - ####  [Note] install_complex_pdf will take about 30 minutes.

In [3]:
install_needed = True
install_complex_pdf = False

In [4]:
import os
import sys
import IPython
import subprocess

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    #!{sys.executable} -m pip install -U spacy
    #!{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    #!{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu
    !{sys.executable} -m pip install -U jq
    !{sys.executable} -m pip install -U pydantic

    if install_complex_pdf:
        
        response = subprocess.run(['cat', '/etc/os-release'], capture_output=True)
        response = response.stdout.decode("utf-8")
        
        if "Amazon Linux" in response: ## SageMaker Notebook

            !sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
            !sudo yum -y update
            !sudo yum install -y poppler-utils
            !{sys.executable} -m pip install -U lxml
            !{sys.executable} -m pip install -U kaleido
            !{sys.executable} -m pip install -U uvicorn
            !{sys.executable} -m pip install -U pandas
            !{sys.executable} -m pip install -U numexpr
            !{sys.executable} -m pip install -U pdf2image
            !sudo sh install_tesseract.sh
            !sudo amazon-linux-extras install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !sudo rm -rf leptonica-1.84.1 leptonica-1.84.1.tar.gz tesseract-ocr

        else: ## SageMaker Studio
            !sudo apt-get install software-properties-common -y
            !sudo add-apt-repository ppa:alex-p/tesseract-ocr5 -y
            !sudo apt-get update -y
            !sudo apt-get install poppler-utils tesseract-ocr -y
            !sudo apt-get install libgl1-mesa-glx libglib2.0-0 -y
            !sudo apt install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !{sys.executable} -m install -U pdf2image
            !cd /usr/share/tesseract-ocr/5/tessdata/ && sudo wget https://github.com/tesseract-ocr/tessdata_best/raw/main/kor.traineddata

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 67.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.8 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
  Attempting uninstall: awscli
    Found existing installation: awscli 1.32.69
    Uninstalling awscli-1.32.69:
      Successfully uninstalled awscli-1.32.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.214.0 requires importlib-metadata<7.0,>=1.4.0, but you have importlib-metadata 7.0.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.69
    Unins

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [5]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [6]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.1.13
boto3 version check: 1.34.69
botocore version check: 1.34.69
awscli version check: 1.32.69


In [7]:
bedrock_region=boto3.Session().region_name

In [8]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [9]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 5. generation

In [11]:
import base64
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### Titan

In [ ]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [ ]:
print (colored(llm(prompt), "green"))

### Claude v2.1

In [18]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f8b7ce3d870>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens': 1024, 'temperature': 0, 'top_p': 0.9, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f8b7c788670>])

In [19]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to ask you a question.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [20]:
response = chain.invoke(
    {
        "question": "세이지메이커의 장점을 100단어로 알려주세요."
    }
)

<answer>
세이지메이커의 장점을 100단어로 알려드리겠습니다. 

세이지메이커는 대화형 AI로, 사용자의 질문에 최적화된 답변을 실시간으로 제공합니다. 풍부한 지식 그래프와 딥 러닝 기술을 바탕으로 정확성과 유연성이 뛰어납니다. 
또한 사용자 피드백을 기반으로 지속적으로 학습하고 업데이트되기 때문에 성능이 점점 향상됩니다. 
다양한 도메인의 대화에 적용 가능하며, 친절하고 유쾌한 대화를 제공할 수 있습니다.
</answer>

In [21]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to ask you a question.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the table as html: <table>{table}</table>
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [24]:
system_prompt = '''
                너의 임무는 2024년 AWS Summit Seoul의 Gen AI Workshop 고객 모집 메일을 작성하는 것이다.
                나는 너에게 몇가지 keyword와 guide를 줄 것이다.
                guide 에 맞게 메일을 써라.
                keyword 들은 반드시 들어가야 한다.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the keyword: <keyword>1. 고객이 workshop 등록을 한다고 해서 workshop에 반드시 참석할 수 있는 것은 아니다. \n2. 비즈니스 옵티가 있는 고객이 우선순위가 있다. \n 3.account당 최대 3명 참석가능하다. \n 고객에 더 많은 기회제공을 위해 파트너 사는 제외한다.</keyword>
                Here is the guide: <guide>{guide}</guide>

                Answer in Korean.
                정중하게 써주세요.
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [26]:
response = chain.invoke(
    {
        "guide": "Account Manager들에게 AWS Summit Seoul의 Gen AI Workshop 안내을 요청하는 메일을 작성해 주세요. 반드시 keyword들이 포함될 수 있도록 써주세요."
    }
)

안녕하세요. AWS Summit Seoul의 Gen AI Workshop 안내 메일을 보내 드리고자 합니다. 

Workshop 정원이 제한되어 있어 참석 가능 여부를 미리 확인해 주시기 바랍니다. 고객사의 비즈니스 우선순위에 따라 참석 인원이 결정될 수 있음을 양해 부탁드립니다. 

Account 당 최대 3명까지 참석 가능하며, 파트너사 직원분들은 참석 기회를 고객사에 더 많이 제공하기 위해 제외될 수 있사오니 양해 부탁드립니다.  

참석 희망하시는 분들은 00월 00일까지 등록을 부탁드리겠습니다. 많은 관심과 성원 부탁드립니다.

In [22]:
response = chain.invoke(
    {
        "table": '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n      <th>2</th>\n      <th>3</th>\n      <th>4</th>\n      <th>5</th>\n      <th>6</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td></td>\n      <td></td>\n      <td>구 분</td>\n      <td>당분기\\n(2023.3.31)</td>\n      <td></td>\n      <td>전년동기\\n(2022.3.31)</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td></td>\n      <td></td>\n      <td></td>\n      <td>금 액</td>\n      <td>구성비</td>\n      <td>금 액</td>\n      <td>구성비</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>자\\n산</td>\n      <td>현금 및 예치금</td>\n      <td></td>\n      <td>24,644</td>\n      <td>0.93</td>\n      <td>25,948</td>\n      <td>0.93</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td></td>\n      <td>당기손익-공정가치측정유가증권</td>\n      <td></td>\n      <td>116,339</td>\n      <td>4.38</td>\n      <td>0</td>\n      <td>0.00</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td></td>\n      <td>당기손익인식금융자산</td>\n      <td></td>\n      <td>0</td>\n      <td>0.00</td>\n      <td>13,929</td>\n      <td>0.50</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td></td>\n      <td>기타포괄손익-공정가치측정유가증권</td>\n      <td></td>\n      <td>1,555,915</td>\n      <td>58.59</td>\n      <td>0</td>\n      <td>0.00</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td></td>\n      <td>매도가능금융자산</td>\n      <td></td>\n      <td>0</td>\n      <td>0.00</td>\n      <td>1,792,754</td>\n      <td>64.55</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td></td>\n      <td>만기보유금융자산</td>\n      <td></td>\n      <td>0</td>\n      <td>0.00</td>\n      <td>2,429</td>\n      <td>0.09</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td></td>\n      <td>상각후원가측정유가증권</td>\n      <td></td>\n      <td>0</td>\n      <td>0.00</td>\n      <td>0</td>\n      <td>0.00</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td></td>\n      <td>관계∙종속기업투자주식</td>\n      <td></td>\n      <td>135,622</td>\n      <td>5.11</td>\n      <td>99,576</td>\n      <td>3.59</td>\n    </tr>\n    <tr>\n      <th>10</th>\n      <td></td>\n      <td>대출채권</td>\n      <td></td>\n      <td>482,281</td>\n      <td>18.16</td>\n      <td>483,311</td>\n      <td>17.40</td>\n    </tr>\n    <tr>\n      <th>11</th>\n      <td></td>\n      <td>부동산</td>\n      <td></td>\n      <td>37,810</td>\n      <td>1.42</td>\n      <td>38,847</td>\n      <td>1.39</td>\n    </tr>\n    <tr>\n      <th>12</th>\n      <td></td>\n      <td>비운용자산</td>\n      <td></td>\n      <td>32,583</td>\n      <td>1.23</td>\n      <td>26,012</td>\n      <td>0.94</td>\n    </tr>\n    <tr>\n      <th>13</th>\n      <td></td>\n      <td>특별계정자산</td>\n      <td></td>\n      <td>270,406</td>\n      <td>10.18</td>\n      <td>294,570</td>\n      <td>10.61</td>\n    </tr>\n    <tr>\n      <th>14</th>\n      <td>자산총계</td>\n      <td></td>\n      <td></td>\n      <td>2,655,600</td>\n      <td>100.00</td>\n      <td>2,777,376</td>\n      <td>100.00</td>\n    </tr>\n    <tr>\n      <th>15</th>\n      <td>부채\\n및\\n자본</td>\n      <td></td>\n      <td>책임준비금</td>\n      <td>1,861,534</td>\n      <td>80.27</td>\n      <td>1,693,867</td>\n      <td>69.03</td>\n    </tr>\n    <tr>\n      <th>16</th>\n      <td></td>\n      <td></td>\n      <td>계약자지분조정</td>\n      <td>67,029</td>\n      <td>2.89</td>\n      <td>95,989</td>\n      <td>3.91</td>\n    </tr>\n    <tr>\n      <th>17</th>\n      <td></td>\n      <td></td>\n      <td>기타부채</td>\n      <td>120,139</td>\n      <td>5.18</td>\n      <td>369,418</td>\n      <td>15.06</td>\n    </tr>\n    <tr>\n      <th>18</th>\n      <td></td>\n      <td></td>\n      <td>특별계정부채</td>\n      <td>270,427</td>\n      <td>11.66</td>\n      <td>294,325</td>\n      <td>12.00</td>\n    </tr>\n    <tr>\n      <th>19</th>\n      <td></td>\n      <td>부채총계</td>\n      <td></td>\n      <td>2,319,129</td>\n      <td>100.00</td>\n      <td>2,453,599</td>\n      <td>100.00</td>\n    </tr>\n    <tr>\n      <th>20</th>\n      <td></td>\n      <td>자본총계</td>\n      <td></td>\n      <td>336,471</td>\n      <td></td>\n      <td>323,777</td>\n      <td></td>\n    </tr>\n    <tr>\n      <th>21</th>\n      <td>부채 및 자본총계</td>\n      <td></td>\n      <td></td>\n      <td>2,655,600</td>\n      <td></td>\n      <td>2,777,376</td>\n      <td></td>\n    </tr>\n  </tbody>\n</table>',
        "question": "부동산의 전년동기 및 당분기 구성비는?"
    }
)

<answer>
전년동기(2022.3.31) 부동산 구성비: 1.39%
당분기(2023.3.31) 부동산 구성비: 1.42%
</answer>

### Claude v3 sonnet and haiku
## test: 아래 이미지에 대한 질문
![nn](../imgs/setup/attention-is-all-you-needs.png)

In [ ]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"), # or "Claude-V3-Sonnet"
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm

In [ ]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

In [ ]:
with open("../imgs/setup/attention-is-all-you-needs.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')
    
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{base64_string}",
        },
    },
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke(
    {
        "question": "MoE 모델의 EN-FR BLEU 스코어는 무엇입니까?"
    }
)